<a href="https://colab.research.google.com/github/jis478/Tensorflow/blob/master/StarGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install tensorflow-gpu==2.0.0-alpha0

In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-alpha0


In [135]:
# Library loading
import tensorflow as tf
import numpy as np 
from tensorflow.keras.layers import Dense, Flatten, Conv2D, ReLU

# Data generation
N = 1000
image_size = 128
image_data = np.random.normal(size=[N, image_size, image_size, 3])

label_1 = np.random.uniform(low=0., high=3., size=N).astype(np.int32) # facial expression attributes
label_1 = tf.one_hot(label_1, depth=3)
label_2 = np.random.randint(2, size=(N,2)) # male, young attributes
label =  np.concatenate([label_1, label_2], axis=-1)

train_dataset = tf.data.Dataset.from_tensor_slices((image_data,label))
train_dataset = train_dataset.batch(2) 
images, labels = next(iter(train_dataset))


# Downsampling class
class Downsampling(tf.keras.Model):
  def __init__(self):
    super(Downsampling, self).__init__()
    self.conv1 = Conv2D(64, kernel_size = 7, strides = 1, padding = 'SAME')
    self.conv2 = Conv2D(128, kernel_size = 4, strides = 2, padding = 'SAME')
    self.conv3 = Conv2D(256, kernel_size = 4, strides = 2, padding = 'SAME')
    self.activation = ReLU()
    
  def input_merge(self, images, labels):
    batch_size = images.shape[0]
    image_size = images.shape[1]
    channels = labels.shape[1]
    merged = np.zeros([batch_size,image_size,image_size,channels])
    for batch in range(batch_size):
      temp = tf.broadcast_to(labels[batch], [image_size,image_size,channels])
      merged[batch] = temp
    merged = tf.concat([images, merged], axis=-1)
    return merged  
  
  def IN(self, images):
    HW = images.shape[1]*images.shape[2] 

    u_ti = 1/HW*np.sum(images, axis=(1,2))   # 2x3  
    for _ in range(2):
      u_ti = np.stack((u_ti, )*images.shape[1], axis=-1) # 2x3x128x128
    u_ti = np.swapaxes(u_ti,1,3) # 2x128x128x3  

    var_ti = 1/HW*np.sum((images - u_ti), axis=(1,2))**2 # 2x3  
    for _ in range(2):
      var_ti = np.stack((var_ti, )*images.shape[1], axis=-1) # 2x3x128x128
    var_ti = np.swapaxes(var_ti,1,3) # 2x128x128x3                      

    y_tijk = (images - u_ti) / np.sqrt(var_ti + np.finfo(np.float32).eps)  # 2x128x128x3
    return y_tijk   
    

  def call(self, images, labels):
    x = self.input_merge(images,labels)
    x = self.conv1(x)
    x = self.IN(x)
    x = self.activation(x)
    assert x.shape == (2, 128, 128, 64) 

    x = self.conv2(x)
    x = self.IN(x)
    x = self.activation(x)
    assert x.shape == (2, 64, 64, 128) 
    
    x = self.conv3(x)
    x = self.IN(x)
    x = self.activation(x)
    assert x.shape == (2, 32, 32, 256) 
    return x
  
# Execution  
Downsampling_layer = Downsampling()
result = Downsampling_layer(images, labels)
print("The shape of the input tensor after downsampling :", result.shape)
result


The shape of the input tensor after downsampling : (2, 32, 32, 256)


<tf.Tensor: id=397, shape=(2, 32, 32, 256), dtype=float64, numpy=
array([[[[0.00000000e+00, 3.57157220e+09, 0.00000000e+00, ...,
          1.85816865e+09, 2.69338984e+08, 1.38279445e+09],
         [3.56418879e+07, 2.90105219e+09, 0.00000000e+00, ...,
          3.55210667e+08, 2.01552436e+09, 0.00000000e+00],
         [0.00000000e+00, 4.41503094e+08, 3.94627208e+08, ...,
          2.65844097e+09, 0.00000000e+00, 0.00000000e+00],
         ...,
         [7.95467666e+08, 1.70616674e+09, 0.00000000e+00, ...,
          1.07187737e+09, 1.53128227e+09, 0.00000000e+00],
         [1.79243663e+09, 0.00000000e+00, 1.24221011e+09, ...,
          1.00235935e+09, 4.55185853e+08, 0.00000000e+00],
         [0.00000000e+00, 1.13915369e+09, 0.00000000e+00, ...,
          1.15747940e+09, 5.19286077e+08, 2.13802626e+09]],

        [[0.00000000e+00, 2.12947722e+09, 0.00000000e+00, ...,
          3.42995301e+07, 0.00000000e+00, 2.40386568e+09],
         [0.00000000e+00, 0.00000000e+00, 2.82249365e+09, ...,
 